In [ ]:
import torch as th
import matplotlib.pyplot as plt

from music_diffusion.networks import Denoiser
from music_diffusion.data import magnitude_phase_to_wav

In [ ]:
d = Denoiser(
    2, 4096, 48, 2.5e-5, 5e-3, [(16,32),(32,48),(48,64),(64,80)], 16
)
ema_prefix = "ema_model."
state_dict = {
            k[len(ema_prefix) :]: p
            for k, p in th.load("/media/samuel/M2_Sam/train_diffusion_bach-stride128-16kHz_ema-hybrid_5/denoiser_ema_168.pt").items()
            if k.startswith(ema_prefix)
        }

d.load_state_dict(state_dict)
d.cuda()
d.eval()

In [ ]:
d.count_parameters()

In [ ]:
x_t = th.randn(5, 2, 512, 512, device="cuda")

In [ ]:
with th.no_grad():
    x_0 = d.sample(x_t, verbose=True)

In [ ]:
x_0.size()

In [ ]:
for i in range(x_0.size(0)):
    print(x_0[i, 0].min(), x_0[i, 0].max(), x_0[i, 0].mean())

In [ ]:
plt.matshow(x_0[0, 0].cpu().detach(), cmap="plasma")

In [ ]:
x_0 = (x_0 - x_0.min()) / (x_0.max() - x_0.min())
magnitude_phase_to_wav(x_0[0, None].cpu(), "out_super_bach.wav", 16000, 1024, 128)